In [ ]:
#!python3 -m pip install requests
!python3 -m pip freeze

In [1]:
import cv2
import requests
import numpy as np
import base64
import json
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from grpc.beta import implementations
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
print(os.path.dirname(implementations.__file__))

/usr/local/lib/python3.5/dist-packages/grpc/beta


In [2]:
input_name = 'inputs'
def resize_input_char(im):
    h, w, c = [112,112,3]
    imsz = cv2.resize(im, (w, h))
    imsz = imsz / 255.
    imsz = imsz[:,:,::-1]
    return imsz

def prepare_grpc_request(model_name, signature_name, data):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.model_spec.signature_name = signature_name
    request.inputs[input_name].CopyFrom(
        tf.contrib.util.make_tensor_proto(data, dtype=None))
    return request

In [3]:
input_batch = []
model_num= "yolo_num_1"
model_char= "yolo_char_1"
signature_name= "predict_signature"
port_num = '8501'
port_char = '8503'
host_num = '172.17.0.8'
host_char = '172.17.0.13'
batch_size=1
path1 = "/workspace/ALPR/training_grounds/recognition/source/1000_7_num.jpg"
path2 = "/workspace/ALPR/training_grounds/recognition/source/111.jpg"
image_num = cv2.imread(path1)
image_char = cv2.imread(path2)
_, input_num = cv2.imencode(".jpg",image_num, ([int(cv2.IMWRITE_JPEG_QUALITY),100]))
_, input_char = cv2.imencode(".jpg",image_char, ([int(cv2.IMWRITE_JPEG_QUALITY),100]))
#input_np = np.expand_dims(resize_input_char(image_input),0)
#input_batch.append(input_cd)
#input_batch.append(input_cd)
#batch = np.repeat(input_np, 1, axis=0).tolist()

In [4]:
json = {
    "signature_name": signature_name,
    "instances": batch
}

NameError: name 'batch' is not defined

In [5]:
response = requests.post("http://%s:%s/v1/models/%s:predict" % (host, port, model_name), json=json)

In [6]:
print(response.text)

{ "error": "Failed to process element: 0 of \'instances\' list. Error: Invalid argument: JSON Value: 0.0 Type: Number is not of expected type: string" }


In [14]:
a = response.json()
print(a["predictions"])

[[4.09967e-07, 3.65579e-05, 2.39485e-06, 2.52607e-07, 6.03311e-07, 3.48869e-06, 1.86636e-07, 0.999956, 2.5209e-07, 2.31659e-07]]


In [4]:
batch = np.repeat(input_np, 1, axis=0).tolist()    

In [4]:
#batch = [{"b64": list(base64.b64encode(np.repeat(input_np, batch_size, axis=0)))}]
batch_num = [{"b64": base64.b64encode(input_num).decode("utf-8")}]
batch_char = [{"b64": base64.b64encode(input_char).decode("utf-8")}]
#batch_for_json = [{"inputs":{"b64": base64.urlsafe_b64encode(bytes(input_cd)).decode("utf-8")}}]
#ser = base64.b64encode(np.array(x)).decode("utf-8")
_json_num = {
    "signature_name": signature_name,
    "instances": batch_num
}
_json_char = {
    "signature_name": signature_name,
    "instances": batch_char
}

In [7]:
response_num = requests.post("http://%s:%s/v1/models/%s:predict" % (host_num, port_num, model_num), json=_json_num)
response_char = requests.post("http://%s:%s/v1/models/%s:predict" % (host_num, port_num, model_char), json=_json_char)

In [8]:
print(response_num.text)
print(response_char.text)
#print(response.json())

{
    "predictions": [[8.16848e-06, 0.000106764, 4.8675e-06, 1.13831e-06, 4.54782e-06, 2.23841e-06, 5.99176e-07, 0.999869, 1.43953e-06, 1.69876e-06]
    ]
}
{
    "predictions": [[0.978321, 2.94785e-06, 1.94898e-05, 1.75677e-06, 1.24997e-05, 1.25228e-05, 4.38197e-06, 0.000160846, 7.46149e-06, 0.000526133, 5.80359e-05, 0.0197628, 0.000161974, 0.000548948, 3.35241e-05, 8.19394e-06, 2.8283e-06, 4.03734e-06, 3.05423e-06, 4.10535e-05, 1.80634e-05, 1.4748e-06, 4.75788e-05, 0.000128829, 2.44211e-06, 0.000107787]
    ]
}


In [8]:
print(np.argmax(response_num.json()))
print(np.argmax(response_char.json()))

0
0


In [13]:
grpc_port = "8500"
channel = implementations.insecure_channel(host, int(grpc_port))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
request = prepare_grpc_request(model_name, signature_name, [bytes(input_cd),bytes(input_cd)])

In [16]:
print(request)
response = stub.Predict(request, timeout=600)

model_spec {
  name: "yolo_num_1"
  signature_name: "predict_signature"
}
inputs {
  key: "inputs"
  value {
    dtype: DT_STRING
    tensor_shape {
      dim {
        size: 2
      }
    }
    string_val: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\377\333\000C\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\001\377\300\000\021\010\0008\0008\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\0

In [17]:
print([response.outputs['outputs'].float_val])

[[6.016711631673388e-05, 5.961901115369983e-05, 0.00015766175056342036, 0.0008133560768328607, 4.471688225748949e-05, 0.3266528844833374, 6.216298788785934e-05, 0.012203169986605644, 0.00017959260731004179, 3.681830639834516e-05, 0.0006568736280314624, 5.305330705596134e-05, 0.0029284018091857433, 0.0009367761667817831, 0.00011548202746780589, 0.06592676043510437, 3.025297337444499e-05, 0.002135596005246043, 0.00033291763975284994, 0.2055499106645584, 8.70025614858605e-05, 0.0021809341851621866, 0.00020153116201981902, 9.712013707030565e-05, 0.0046633584424853325, 0.37383389472961426, 6.016711631673388e-05, 5.961901115369983e-05, 0.00015766175056342036, 0.0008133560768328607, 4.471688225748949e-05, 0.3266528844833374, 6.216298788785934e-05, 0.012203169986605644, 0.00017959260731004179, 3.681830639834516e-05, 0.0006568736280314624, 5.305330705596134e-05, 0.0029284018091857433, 0.0009367761667817831, 0.00011548202746780589, 0.06592676043510437, 3.025297337444499e-05, 0.002135596005246043